<h1 style="text-align: center"> 实体消歧示范教程 </h1>

## 1、实体消歧的教程概述
- **任务描述**：本文介绍实体消歧的概念、方法及实现。实体消歧的目的是解决同名实体产生的歧义问题。在实际的语言环境中，大量存在一个实体提及（entity mention）对应多个命名实体对象的现象，即一词多义，因此需要通过实体消歧明确实体提及具体指代的实体，如下例：

|text|entity|entity desc|
|:---|:---|:---|
|苹果手机放上卡怎么读卡里的电话号码？|苹果|苹果公司（Apple Inc. ）是美国一家高科技公司。|
|曾经鼎鼎大名的秦冠苹果你吃过吗？|苹果|苹果是蔷薇科苹果亚科苹果属植物，其树为落叶乔木。|
|影片《苹果》讲述了善良的贤正陷入了两难的选择之中的故事|苹果|《苹果》是由康理贯执导，文素丽等人主演的韩国爱情电影|

- **数据集**：本案例使用 CCKS 2020 面向中文短文本的实体链指任务数据集。[CCKS 2020 EL评测任务网站](https://www.biendata.xyz/competition/ccks_2020_el/)、数据可由此[下载](https://drive.google.com/file/d/1RI-K_Sy8EzP2R7_Vlp3jh6ISHRSrlhJ4/view?usp=sharing)。该数据包含来自百度百科知识库和来自互联网的短文本标注数据集。本案例使用预训练的中文词向量，开源项目网址：[Chinese-Word-Vectors](https://github.com/Embedding/Chinese-Word-Vectors)，数据采用以百度百科为语料，训练的 300D 字词向量，[下载地址](https://drive.google.com/file/d/1tArKdGCgFUbeXYZn6onLbrxb430CiZyq/view?usp=sharing)。

<img src="https://s1.ax1x.com/2020/07/28/aVPp6S.png" alt="word2vec.png" border="0" align=center style="zoom: 50%;"/>  

- **运行环境**：在Python3.7环境下测试了本教程代码。需要的第三方模块和版本包括：

```
pytorch=1.5.1
json=2.0.9
csv=1.0
jieba=0.39
pickle=4.0
tqdm=4.47.0
```
```
numpy=1.18.5
可以使用 pip 命令安装上述模块： 
pip install torch torchvision
pip install json, cav, jieba, pickle, tqdm, numpy
```
- **方法概述**：根据是否给定目标实体列表（知识库），实体消歧的方法大致分为两类：基于聚类的实体消歧和基于实体链接的实体消歧。本文将构建一个基于实体链接的消歧系统。通常端到端的实体消歧系统需要完成**命名实体识别**和**实体链接**两个任务。为了专注于文本的歧义消除技术，本案例采用标注出的实体，不再进行实体识别的工作。**实体链接有三个步骤：1、产生候选实体；2、对候选实体排序；3、无法链接预测。**产生候选实体就是构建候选实体集合，本案例通过知识库的 alias 字段构建候选实体字典，候选实体排序采用深度学习方法，无法链接预测是对没有出现在知识库中的新实体，预测其类型。

- **建议学习时长**：120分钟  

- **<span class="mark">数据集版权说明：本实践使用互联网开源数据集，请勿用于商业用途。</span>**

In [1]:
# 使用 pip 命令安装指定版本的工具包
# !pip install torch torchvision
# !pip install json, cav, jieba, pickle, tqdm, numpy

In [3]:
# 导入必要的模块
import os
import json, csv
import pickle
import jieba
import random
import string
from tqdm import tqdm

import numpy as np

import torch
import torch.nn as nn
import torch.utils.data as DATA
from torch.nn.utils.rnn import pad_packed_sequence
from torch.nn.utils.rnn import pack_padded_sequence


## 2、数据处理
（1） 数据说明

（2） 处理知识库

（3） 处理标注数据集

（4） 数据编码

（5） 构建数据加载接口

### 2.1、数据说明

- **数据整理**：
    - 将下载的训练数据解压后放在 `./datasets/basic_data/` 目录下，其中包含以下文件
    ```
    kb.json:知识库文件
    train.json:训练集
    dev.json:验证集
    test.json:测试集
    ```
    - 将下载的词向量下载后放在 `./datasets/pretrain_data` 目录下，其中包含 `word2vec.iter5` 文件
    文件格式如下：
    ```
    word_num, vec_dim
    word1, vec_dim1, vec_dim2, ……
    word2, vec_dim1, vec_dim2, ……
    word2, vec_dim1, vec_dim2, ……
    ……
    ```
- **数据描述**：

    - **知识库**：该任务知识库来自百度百科知识库。知识库中的每个实体都包含一个 subject_id (知识库 id)，一个 subject 名称，实体的别名，对应的概念类型，以及与此实体相关的一系列二元组< predicate，object>（<属性，属性值>）信息形式。知识库中每行代表知识库的一条记录（一个实体信息），每条记录为 json 数据格式。示例如下：
```json
{
    "subject_id":"1000131",
    "subject":"小王子",
    "alias":["Le Petit Prince","The Little Prince","リトルプリンス 星の王子さまと私","小王子"],
    "type":["Work"],
    "data":[
        {"predicate":"外文名","object":"Le Petit Prince"},
        {"predicate":"发行公司","object":"派拉蒙影业"},
        {"predicate":"类型","object":"奇幻"}
    ]
}
```
    - **标注数据集**：标注数据集由训练集、验证集和测试集组成，整体标注数据大约10万条左右，按8:1:1比例分配。标注数据集主要来自于：真实的互联网网页标题数据、视频标题数据、用户搜索 query。标注数据集中每条数据的格式如下：
```json
{
    "text_id":"1",
    "text":"《琅琊榜》海宴_【原创小说|权谋小说】",
    "mention_data":[
        {"kb_id":"2135131","mention":"琅琊榜","offset":"1"},
        {"kb_id":"10572965","mention":"海宴","offset":"5"},
        {"kb_id":"215143","mention":"原创小说","offset":"9"},
        {"kb_id":" NIL_Work ","mention":"权谋小说","offset":"14"}
    ]
}
```
数据集包含24种实体：

|Type| 中文名 |Type| 中文名 |Type| 中文名 |
|:---|:---|:---|:---|:---|:---|
|Event|	事件活动|Vehicle|	车辆|Other|	其他|
|Person|	人物|Website|	网站平台|Software|	软件|
|Work|	作品|Disease&Symptom|	疾病症状|Medicine|	药物
|Location|	区域场所|Organization|	组织机构|Culture|	文化|
|Time&Calendar|	时间历法|Awards|奖项|VirtualThings|	虚拟事物|
|Brand|	品牌|Education|	教育|Diagnosis&Treatment|	诊断治疗方法|
|Natural&Geography|	自然地理|Constellation|	星座|Food|	食物|
|Game|	游戏|Biological|	生物|Law&Regulation|	法律法规|


### 2.2、处理知识库
处理知识库，构建一个由实体提及映射到相似实体的候选字典 cand_dic、知识库实体字典 ent_dic。
其中：  
1. cand_dic 以知识库中的实体提及 mention 为 key，以候选实体 subject_id 集合为 values：`{mention: [id_list]}`； 候选实体字典序列化，保存到 `./datasets/generated/cand.pkl` 
2. ent_dic 以实体的 subject_id 为 key，以实体描述 ent_desc、实体类型 subject_type 为 values：`{kb_id, <ent_name, ent_desc, type>}` 实体字典序列化，保存到 `./datasets/generated/entity.pkl` 

In [4]:
class GenerateCand(object):
    """生成候选实体字典"""

    def __init__(self):
        self.cand_dic = {}  # 候选实体集合 {mention: [id_list]}
        self.ent_dic = {}  # 实体字典 {kb_id：<entity, type>}

    def interface(self, file_path):
        """interface"""
        for line in open('./datasets/basic_data/' + file_path, 'r', encoding='utf-8'):
            line_json = json.loads(line.strip())
            mention_list = line_json.get('alias')
            subject_name = line_json.get('subject')
            mention_list.append(subject_name)
            subject_id = line_json.get('subject_id')
            subject_type = line_json.get('type')
            ent_desc = ''
            for item in line_json.get('data'):
                ent_desc += item.get('predicate') + ':' + item.get('object') + ';'
            # generate ent_dic
            self.ent_dic[subject_id] = {}
            self.ent_dic[subject_id]['ent_name'] = subject_name
            self.ent_dic[subject_id]['ent_desc'] = ent_desc
            self.ent_dic[subject_id]['type'] = subject_type
            # generate cand_dic
            for mention in mention_list:
                if mention in self.cand_dic:
                    self.cand_dic[mention]['iid_list'].append(subject_id)
                    self.cand_dic[mention]['iid_list'] = list(
                        set(self.cand_dic[mention]['iid_list']), )
                else:
                    self.cand_dic[mention] = {}
                    self.cand_dic[mention]['iid_list'] = [subject_id]
        # generate entity mention list         
        with open("./datasets/generated/mention.txt", 'w', encoding='utf-8') as f:
            for mention in self.cand_dic.keys():
                f.write(str(mention) + '\n')
            
        pickle.dump(self.cand_dic, open('./datasets/generated/' + 'cand.pkl', 'wb'))
        pickle.dump(self.ent_dic, open('./datasets/generated/' + 'entity.pkl', 'wb'))
        return self.cand_dic, self.ent_dic

    def load_data(self, file_path):
        """load generated candidate dict"""
        self.cand_dic = pickle.load(open('./datasets/generated/' + 'cand.pkl', 'rb'))
        self.ent_dic = pickle.load(open('./datasets/generated/' + 'entity.pkl', 'rb'))
        return self.cand_dic, self.ent_dic


generate_cand = GenerateCand()
if os.path.exists('./datasets/generated/cand.pkl'):
    cand_dic, ent_dic = generate_cand.load_data('kb.json')
else:
    cand_dic, ent_dic = generate_cand.interface('kb.json')

### 2.3、处理标注数据集

1、加载原始数据，针对每个 mention 匹配出候选实体集合： `cand`   
  
2、构建排序数据，本案例采用排序学习（LTR，learning to rank）算法中的 pairwise 模型：**pairwise 模型是指对于一个 query，将所有 cand_desc 两两组成一个pair，比如 (X1,X2) ,如果 X1 的分值 (与 query 的相似度) 大于 X2 则将该 pair 当作正例 1，否则为负例 0；从而对 pairwise 模型进行学习。** 因此需要将数据集处理成 `query - <entity_pairs>` 的数据格式。  
  
3、将原始数据处理后，生成新的数据文件：`train_data.txt`、`dev_data.txt`、`test_data.txt`  
  
>注意：对于不能被链接到知识库的 mention，原始数据集中以 NIL_<type\> 的形式进行标注，因此本案例将 NIL 作为知识库的一种实体，同时预测所有 query 中的每一个 mention 的实体类型（24种），因此，对于不能链接到知识库的 mention 其 cand_id = ‘NIL’。

生成训练数据 `is_train = True`，格式如下：  

|text_id | mention_id | query | mention| cand1_desc | cand2_desc | cand1_id | cand2_id | lable | type | kb_id |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|2|1|永嘉厂房出租……|厂房|摘要:无隔墙的房屋……|摘要:工业生产用房，……|40189|252045|	0|Location|252045|

生成测试数据 `is_train = False`，格式如下：

|text_id | mention_id | query | mention | cand_desc | cand_id | type | kb_id |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|12|2|宝可梦经典依旧：有趣的童年回忆……|回忆|摘要:《回忆》是南征北战NZBZ创作并演唱的歌曲;……|111847|None|None|


In [4]:
class DataProcess(object):
    """data process"""
    
    def interface(self, file_path, cand_dic, ent_dic, is_train=True):
        # 保存训练数据
        data_fname = file_path.replace('.json', '_data.txt')
        out_file = open('./datasets/generated/' + data_fname, 'w', encoding='utf-8')
        # 处理数据，生成查询文本(query)和候选实体对(ent_pair)数据
        for line in open('./datasets/basic_data/' + file_path, encoding='utf-8'):
            line_json = json.loads(line.strip())
            text_id = line_json.get('text_id')
            query = line_json.get('text')
            mention_data = line_json.get('mention_data')
            mention_id = 0
            for item in mention_data:
                mention = item.get('mention')
                offset = item.get('offset')
                # 获取最佳匹配结果
                kb_id = item.get('kb_id', 'None')
                if 'NIL' in kb_id:
                    golden_desc = mention
                    if '|' in kb_id:
                        kb_id = kb_id.split('|')[0]
                    golden_type = kb_id.replace('NIL_', '')
                    kb_id = 'NIL'
                elif kb_id is not 'None':
                    golden_desc = ent_dic[kb_id]['ent_desc']
                    golden_type = ent_dic[kb_id]['type']
                    if '|' in golden_type:
                        golden_type = golden_type.split('|')[0]
                else:
                    golden_type = 'None'
                # 匹配候选实体集
                if mention in cand_dic:
                    cand = cand_dic[mention]
                    iid_list = cand['iid_list']
                elif not is_train:
                    cand = {}
                    iid_list = []
                else:
                    continue
                # 将 NIL 添加到每一个 mention 的候选实体集合中
                iid_list.append('NIL')
                iid_list = list(set(iid_list))
                cand['iid_list'] = iid_list
                # 生成 ent_pair 实体对 (pairwise 模型)
                for iid in iid_list:
                    if iid != 'NIL':
                        tmp_desc = ent_dic[iid]['ent_desc']
                    else:
                        tmp_desc = mention
                    if not is_train:
                        out_file.write(text_id + '\t' + str(mention_id) + '\t' +
                                       query + '\t' + mention + '\t' + tmp_desc + 
                                       '\t' + iid + '\t' + golden_type + '\t' + kb_id + '\n')
                        continue
                    elif iid == kb_id:
                        continue

                    # pair 随机生成正样本（前面的相似度大于后面）和负样本（反之）
                    random_threshold = random.random()
                    if random_threshold > 0.5:
                        out_file.write(text_id + '\t' + str(mention_id) + '\t' +
                                       query + '\t' + mention + '\t' +
                                       golden_desc + '\t' + tmp_desc + '\t' +
                                       kb_id + '\t' + iid + '\t' + '1' + '\t' +
                                       golden_type + '\t' + kb_id + '\n')
                    else:
                        out_file.write(text_id + '\t' + str(mention_id) + '\t' +
                                       query + '\t' + mention + '\t' +
                                       tmp_desc + '\t' + golden_desc + '\t' +
                                       iid + '\t' + kb_id + '\t' + '0' + '\t' +
                                       golden_type + '\t' + kb_id + '\n')
                mention_id += 1
        out_file.close()

# dataprocess = DataProcess()
# if not os.path.exists('./datasets/generated/train_data.txt'):
#     dataprocess.interface('train.json', cand_dic, ent_dic, is_train=True)
    
# if not os.path.exists('./datasets/generated/dev_data.txt'):
#     dataprocess.interface('dev.json', cand_dic, ent_dic, is_train=False)
    
# if not os.path.exists('./datasets/generated/test_data.txt'):
#     dataprocess.interface('test.json', cand_dic, ent_dic, is_train=False)

### 2.4、数据编码
1. 根据预训练词向量构建 `word2idx`: 词语到索引的映射；`type2lable`: 类型到标签的映射。
2. 对数据中的文本进行规范化，并编码
3. 将编码文件保存到 `./datasets/generated/` 目录下：`train.csv`、`dev.csv`、`test.csv`

编码主要包括两个内容的编码，（1）文本序列（2）类型标签  
文本序列的编码：  
    - ① 文本分词   
    - ② 过滤标点符号（以及语义信息不明显的中文停用词）  
    - ③ 根据 `word2idx` 将分词后的序列，每一个词替换成索引。    
类型标签的编码：根据 `type2label` 将标签映射到 label

In [5]:
def loadWord2Vec(path):
    vocab_size, size = 0, 0
    vocab = {}
    vocab["i2w"], vocab["w2i"] = [], {}
    count = 0
    with open(path, "r", encoding="utf-8") as f:
        first_line = True
        for line in tqdm(f, desc='Build vocab'):
            if first_line:
                first_line = False
                vocab_size = int(line.strip().split()[0]) + 2
                size = int(line.rstrip().split()[1])
                matrix = np.zeros(shape=(vocab_size, size), dtype=np.float32)
                vocab["w2i"]["<unk>"] = count
                vocab["w2i"]["<pad>"] = count + 1
                matrix[1, :] = np.array([1.0] * size)
                count += 2
                continue
            vec = line.strip().split()
            if not vocab["w2i"].__contains__(vec[0]):
                vocab["w2i"][vec[0]] = count
                matrix[count, :] = np.array([float(x) for x in vec[1:]])
                count += 1
    for w, i in vocab["w2i"].items():
        vocab["i2w"].append(w)
    return matrix, vocab, size, len(vocab["i2w"])

# matrix 向量数组；vocab 包含 vocab["w2i"]: word2idx、vocab["i2w"]：idx2word；向量维度，字词数
matrix, vocab, vec_dim, vocab_size = loadWord2Vec("./datasets/pretrain_data/word2vec.iter5")

# 构建类型到标签的映射字典
type2label = {
    "Other": 0,
    "Person": 1,
    "Work": 2,
    "Culture": 3,
    "Organization": 4,
    "VirtualThings": 5,
    "Location": 6,
    "Education": 7,
    "Website": 8,
    "Software": 9,
    "Game": 10,
    "Medicine": 11,
    "Natural&Geography": 12,
    "Biological": 13,
    "Event": 14,
    "Food": 15,
    "Disease&Symptom": 16,
    "Constellation": 17,
    "Time&Calendar": 18,
    "Brand": 19,
    "Vehicle": 20,
    "Awards": 21,
    "Law&Regulation": 22,
    "Diagnosis&Treatment": 23
}

Build vocab: 636087it [01:30, 7000.76it/s]


In [6]:
class DataEncoder(object):
    """对数据进行编码处理"""
    
    def __init__(self, word2idx, type2label):
        self.word2idx = word2idx
        self.type2label = type2label
        jieba.load_userdict("./datasets/generated/mention.txt")
        
    def tokenize(self, text):
        # jieba 分词
        text = jieba.lcut(text, HMM=True)
        # 去掉标点符号
        return [word for word in text if word not in string.punctuation]
    
    def data_encode(self, fname, is_train=True):
        data_file = open(fname.replace("_data.txt", ".csv"), 'w', encoding='utf-8', newline='')
        writer = csv.writer(data_file)
        with open(fname, 'r', encoding='utf-8') as f:
            if is_train:
                writer.writerow([
                    'text_id', 'mention_id', 'query', 'offset', 'cand1_desc',
                    'cand2_desc', 'cand1_id', 'cand2_id', 'label', 'type', 'golden_id'
                ])
                for line in tqdm(f, desc='Encode ' + fname.split('/')[-1][:-4]):
                    try:
                        line = line.strip().split('\t')
                        query = self.tokenize(line[2])
                        mention = line[3]
                        line[3] = offset = [i for i, x in enumerate(query)if x.find(mention) != -1][0]
                        line[2] = query = [
                            self.word2idx[word]
                            if word in self.word2idx else self.word2idx['<unk>']
                            for word in query
                        ]
                        line[4] = cand1_desc = [
                            self.word2idx[word]
                            if word in self.word2idx else self.word2idx['<unk>']
                            for word in self.tokenize(line[4])
                        ]
                        line[5] = cand2_desc = [
                            self.word2idx[word]
                            if word in self.word2idx else self.word2idx['<unk>']
                            for word in self.tokenize(line[5])
                        ]
                        line[9] = type_id = self.type2label[line[9]]
                        writer.writerow(line)
                    except:
                        continue
            else:
                writer.writerow([
                    'text_id', 'mention_id', 'query', 'offset', 'cand_desc',
                    'cand_id', 'type', 'golden_id'
                ])
                for line in tqdm(f, desc='Encode ' + fname.split('/')[-1][:-4]):
                    try:
                        line = line.strip().split('\t')
                        query = self.tokenize(line[2])
                        mention = line[3]
                        line[3] = offset = [i for i, x in enumerate(query) if x.find(mention) != -1][0]
                        line[2] = query = [
                            self.word2idx[word]
                            if word in self.word2idx else self.word2idx['<unk>']
                            for word in query
                        ]
                        line[4] = cand_desc = [
                            self.word2idx[word]
                            if word in self.word2idx else self.word2idx['<unk>']
                            for word in self.tokenize(line[4])
                        ]
                        line[6] = type_id = self.type2label.get(line[6], -1)
                        writer.writerow(line)
                    except:
                        continue
        data_file.close()
                
# data_encoder = DataEncoder(vocab["w2i"], type2label) 
# if not os.path.exists('./datasets/generated/train.csv'):
#     data_encoder.data_encode("./datasets/generated/train_data.txt", is_train=True)
# if not os.path.exists('./datasets/generated/dev.csv'):
#     data_encoder.data_encode("./datasets/generated/dev_data.txt", is_train=False)
# if not os.path.exists('./datasets/generated/test.csv'):
#     data_encoder.data_encode("./datasets/generated/test_data.txt", is_train=False)

### 2.5、构建数据加载接口
构建一个继承 `torch.utils.data.Dataset` 的数据加载类 DataSet 将数据全部读入内存，并根据索引返回每一条样本数据。需要重载`__len__`、`__getitem__` 方法，`collate_fn` 函数作为 `dataloader` 的函数参数，用以 padding 每个batch内的序列数据，以实现批量训练。

In [7]:
class DataSet(DATA.Dataset):
    """数据集"""
    def __init__(self, path, is_train=True):
        super(DATA.Dataset, self).__init__()
        with open(path, 'r', encoding='utf-8') as file:
            reader = csv.reader(file)
            head = next(reader)
            self.data = [sample for sample in reader]
        self.size = len(self.data)
        self.type_num = 24
        self.is_train = is_train

    def __len__(self):
        return self.size

    def __getitem__(self, item):
        sample = self.data[item]
        if self.is_train:
            # (text_id, mention_id, cand1_id, cand2_id, golden_type, golden_id)
            id_list = (sample[0], sample[1], sample[6], sample[7], sample[9], sample[10])
            query = torch.tensor(json.loads(sample[2]), dtype=torch.long)
            offset = torch.tensor(int(sample[3]))
            cand1_desc = torch.tensor(json.loads(sample[4]), dtype=torch.long)
            cand2_desc = torch.tensor(json.loads(sample[5]), dtype=torch.long)
            label = torch.tensor(int(sample[8]))
            ent_type = torch.tensor(int(sample[9]))
            return id_list, query, offset, cand1_desc, cand2_desc, label, ent_type
        else:
            # (text_id, mention_id, cand_id, golden_type, golden_id)
            id_list = (sample[0], sample[1], sample[5], sample[6], sample[7])
            query = torch.tensor(json.loads(sample[2]), dtype=torch.long)
            offset = torch.tensor(int(sample[3]))
            cand_desc = torch.tensor(json.loads(sample[4]), dtype=torch.long)
            return id_list, query, offset, cand_desc


def collate_fn_train(batch):
    """dataloader 预处理函数参数"""
    max_len_query = 0
    max_len_cand1 = 0
    max_len_cand2 = 0
    batch_size = len(batch)
    len_seq_query,len_seq_cand1, len_seq_cand2 = [], [], []
    for each in batch:
        len_seq_query.append(len(each[1]))
        len_seq_cand1.append(len(each[3]))
        len_seq_cand2.append(len(each[4]))
        if len(each[1]) > max_len_query:
            max_len_query = len(each[1])
        if len(each[3]) > max_len_cand1:
            max_len_cand1 = len(each[3])
        if len(each[4]) > max_len_cand2:
            max_len_cand2 = len(each[4])
    padd_query = torch.LongTensor()
    padd_cand1 = torch.LongTensor()
    padd_cand2 = torch.LongTensor()
    id_list, offset, label, ent_type = [], [], [], []
    # 静态 padding 每个 text 序列到 batch 内最长
    for each in batch:
        tmp1 = torch.ones(max_len_query - len(each[1]), dtype=torch.long)
        tmp2 = torch.ones(max_len_cand1 - len(each[3]), dtype=torch.long)
        tmp3 = torch.ones(max_len_cand2 - len(each[4]), dtype=torch.long)
        padd_query = torch.cat([padd_query, torch.cat([each[1], tmp1])], dim=0)
        padd_cand1 = torch.cat([padd_cand1, torch.cat([each[3], tmp2])], dim=0)
        padd_cand2 = torch.cat([padd_cand2, torch.cat([each[4], tmp3])], dim=0)
        id_list.append(each[0])
        offset.append(each[2])
        label.append(each[5])
        ent_type.append(each[6])
    padd_query = padd_query.view(batch_size, -1)
    padd_cand1 = padd_cand1.view(batch_size, -1)
    padd_cand2 = padd_cand2.view(batch_size, -1)
    label = torch.tensor(label, dtype=torch.float)
    ent_type = torch.tensor(ent_type, dtype=torch.long)
    # 变长序列 query, cand1_desc, cand2desc 的序列长度
    seq_len = (len_seq_query, len_seq_cand1, len_seq_cand2)
    return id_list, padd_query, offset, padd_cand1, padd_cand2, label, ent_type, seq_len

def collate_fn_test(batch):
    """dataloader 预处理函数参数"""
    max_len_query = 0
    max_len_cand = 0
    batch_size = len(batch)
    len_seq_query,len_seq_cand = [], []
    for each in batch:
        len_seq_query.append(len(each[1]))
        len_seq_cand.append(len(each[3]))
        if len(each[1]) > max_len_query:
            max_len_query = len(each[1])
        if len(each[3]) > max_len_cand:
            max_len_cand = len(each[3])
    padd_query = torch.LongTensor()
    padd_cand = torch.LongTensor()
    id_list, offset = [], []
    # 静态 padding 每个 text 序列到 batch 内最长
    for each in batch:
        tmp1 = torch.ones(max_len_query - len(each[1]), dtype=torch.long)
        tmp2 = torch.ones(max_len_cand - len(each[3]), dtype=torch.long)
        padd_query = torch.cat([padd_query, torch.cat([each[1], tmp1])], dim=0)
        padd_cand = torch.cat([padd_cand, torch.cat([each[3], tmp2])], dim=0)
        id_list.append(each[0])
        offset.append(each[2])
    padd_query = padd_query.view(batch_size, -1)
    padd_cand = padd_cand.view(batch_size, -1)
    # 变长序列 query, cand_desc 的序列长度
    seq_len = (len_seq_query, len_seq_cand)
    return id_list, padd_query, offset, padd_cand, seq_len

# train_set = DataSet("./datasets/generated/train_part.csv", is_train=True)
# dev_set = DataSet("./datasets/generated/dev.csv", is_train=False)
# test_set = DataSet("./datasets/generated/test_part.csv", is_train=False)
# matrix, vocab, vec_dim, vocab_size = loadWord2Vec("./datasets/pretrain_data/word2vec.iter5")

## 3、构建候选实体排序 & 类型预测模型

In [8]:
# 模型参数
param = {}
param["batch"] = 64
param["epoch"] = 1
param["lr"] = 0.001
param["loss_w"] = 0.6        # 多任务模型中，总损失 = 排序任务损失(rank_loss) * loss_w + 预测任务损失(classify_loss) * (1 - loss_w)
param["type_num"] = 24       # 实体类别数
param["emb_dim"] = 300       # embedding层词向量维度
param["hidden_dim"] = 150    # BiLSTM 中隐藏层向量维度
param["hidden_dim_fc"] = 150 # 全连接层中隐藏层向量维度
param["resume"] = False      # 模型是否恢复到上次训练状态
param["use_cuda"] = False # torch.cuda.is_available()
param["device"] = torch.device('cuda' if param["use_cuda"] else 'cpu')

In [9]:
# train_loader = DATA.DataLoader(train_set, batch_size=param["batch"], collate_fn=collate_fn_train, drop_last=True)
# dev_loader = DATA.DataLoader(dev_set, batch_size=param["batch"], collate_fn=collate_fn_test, drop_last=True)
# test_loader = DATA.DataLoader(test_set, batch_size=param["batch"], collate_fn=collate_fn_test, drop_last=True)

### 3.1 模型概述
如下图所示，本案例构建一个多任务模型，包含两个任务：候选实体排序和提及类型的预测。  

**候选实体排序任务使用 pairwise 模型。** 在训练阶段，将已经编码的训练预料 `query` 和一对候选实体描述 `<cand1_desc, cand2_desc>` 经过embedding 层进行向量化。将 mention 的位置向量 `mention_offset` 拼接到 query 的向量序列中。将 query、cand1、cand2 通过一层 BiLSTM 提取特征，分别得到序列编码 `Q`、`C1`、`C2`，其中 cand1、cand2 共享 BiLSTM 参数。对 BiLSTM 输出的序列编码（time step）进行 Attention 处理。对于候选实体 cand1, 以 `Q` 为 `Query`, `C1` 为 `Key`、`Values` 做一次 attention，再以 `C1` 为 `Query`, `Q` 为 `Key`、`Values` 做一次 attention，将两次结果相加，再经过全连接层计算出该候选实体 `cand1` 的预测得分 `left_score`。同样以相同的网络结构和网络参数，计算另一个候选实体 `cand2` 的预测得分 `right_score` 。通过 `sigmoid(left_score - right_score)` 比较两个候选实体哪个与提及 mention 更相近，再根据 label数据计算 Loss。

> 注意：pairwise 模型在构建训练数据是，对于每个 pair, 都要有一个 `golden_cand` 和一般候选实体，如果 cand1 与 mention 相似度更高则 label = 1；反之，label = 0 

**提及类型预测使用分类模型。** 在训练阶段，将 query 输入到模型，得到编码序列（与 Rank 任务共享编码）。再经过 self-attention 及全连接层 fc + softmax 预测 mention 类别。结合该 mention 的实际类型计算 Loss。

![](./pictures/model.png)

***BiLSTM 介绍：***  


LSTM （长短期记忆）模型由 RNN 优化而来。RNN 通过引入 “状态” 的概念，记录序列中元素的上下文信息。lstm 引入 “记忆” 的概念，通过遗忘门忘记不重要的记忆；通过输入门，添加新的记忆；通过输出门将当前状态和当前词语的特征输出，并将记忆和状态传递到下一阶段。
$$
f(t) = \sigma(W_f[h_{t-1}, x_t] + b_f)\quad\quad//遗忘门控信号 \\
i_t = \sigma(W_i[h_{t-1}, x_t] + b_i)\quad\quad\quad//输入门控信号 \\
\tilde{C_t} = tanh(W_c[h_{t-1}, x_t] + b_c)\quad//待添加记忆 \\
C_t = f_t*C_{t-1} + i_t*\tilde{C_t}\quad\quad\quad//新的记忆\\
O_t = \sigma(W_o[h_{t-1}, x_t] + b_o)\quad\quad//输出\\
h_t = O_t * tanh(C_t)\quad\quad\quad\quad\quad//新状态
$$
BiLSTM 是双向的 LSTM，即：分别从前到后和从后到前递归计算序列数据，从而将序列的上文和下文信息全部融合到词语的编码中。

<div align="center"><img src="https://i.loli.net/2020/06/11/foCtKGa3hxsgl2B.png" alt="image-20200611004019543" style="zoom:90%;" /></div>


***Attention 介绍：***  
Attention机制的受人类视觉注意力机制启发。人类的视觉在感知东西时，一般不会是一个场景从到头看到尾每次全部都看，往往是根据需求观察注意特定的一部分。而且当我们发现一个场景经常在某部分出现自己想观察的东西时，我们就会进行学习。将来再出现类似场景时把注意力放到该部分上。因此，实质上，Attention 机制其实是一系列注意力分配系数，即对当前输入的加权分配。

本模型中 attention 机制由 attention 函数实现，采用一般的：scaled dot-product attention

分三个步骤：

1. 相似度计算：将query和每个key进行相似度计算得到权重。其中 key 取 BiLSTM 中的输出序列， query 也取 BiLSTM 中的输出序列。相似度计算采用点积。
2. softmax 归一化：将上一步结果进行 softmax 归一化，得到一组 attention weight
3. 对 value 加权求和，value 是 BiLSTM 的输出序列。以attention weight 为权重，对 value 加权求和。

tips： key value 一般设为同一个序列数据，本案例在预测提及类型是采用 self-attention，即：query key value 三者相同，都是 mention desc 经过 BiLSTM 的输出序列。

<div align="center"><img src="https://i.loli.net/2020/06/24/WMPp78tkrjnZw2B.png" style="zoom:67%;" /></div>

***Mask 介绍：*** 

包括 LSTM、GRU 在内的类 RNN 网络不能直接并行处理变长序列，因此不能直接批量训练模型。文本数据往往是变长序列，为了能实现批量训练，需要将变长的文本序列 padding 到相同的长度，在实际训练中，通过 Mask 排除 padding 数据带来的影响。Mask 的实现一般通过 Mask 矩阵实现，即构建一个与 batch 数据前两维形状相同的 0-1 矩阵，用 1 标记实际数据，用 0 标记 padding 数据，通过 Mask 矩阵区分 padding 部分和非 padding 部分。  

在 Pytorch 中，对 Mask 的具体实现形式不是 mask 矩阵，而是通过一个句子长度列表来实现的。具体做法是利用 `torch.nn.utils.rnn` 中的 `pad_packed_sequence`、`pack_padded_sequence` 两个方法实现，其中对于已经 padding 的数据，使用 `pack_padded_sequence` 进行 pack，将 tensor 数据转变成一种 `PackedSequence` 类型数据，通过 LSTM 并行运算后，再通过 `pad_packed_sequence` 还原 packed 数据。

In [10]:
class Model(nn.Module):
    def __init__(self, embd, param):
        super(Model, self).__init__()
        self.param = param
        # 加载预训练的词向量
        embd = torch.from_numpy(embd)
        self.embedding = nn.Embedding.from_pretrained(embd)
        # 将embedding层设置为 “不计算梯度，不进行更新”
        for p in self.parameters():
            p.requires_grad=False
        # 提取查询文本序列（query）特征信息的BiLSTM层
        self.bilstm_query = nn.LSTM(input_size=param["emb_dim"] + 1,
                                    hidden_size=param["hidden_dim"],
                                    batch_first=True,
                                    bidirectional=True,)
        # 提取候选实体描述序列（cand_desc）特征信息的BiLSTM层
        self.bilstm_cand = nn.LSTM(input_size=param["emb_dim"],
                                   hidden_size=param["hidden_dim"],
                                   batch_first=True,
                                   bidirectional=True,)
        # 用于候选实体排序的全连接层
        self.fc_rank = nn.Sequential(
            nn.Linear(param["hidden_dim"] * 2, param["hidden_dim_fc"]),
            nn.ReLU(inplace=True),
            nn.Linear(param["hidden_dim_fc"], 1),
            nn.Sigmoid()
        )
        # 用于实体分类的全连接层
        self.fc_classify = nn.Sequential(
            nn.Linear(param["hidden_dim"] * 2, param["type_num"]),
        )

    def init_hidden(self):
        """ 生成BiLSTM的初始化状态 """
        hidden_state = torch.zeros(1 * 2, self.batch, self.param["hidden_dim"]).to(self.param["device"])
        cell_state = torch.zeros(1 * 2, self.batch, self.param["hidden_dim"]).to(self.param["device"])
        return hidden_state, cell_state

    def Attention(self, q, k, v, scale=None, attn_mask=None):
        """
        desc: scaled dot-product attention
        q: [batch, timestep_q, dim_q]
        k: [batch, timestep_k, dim_k]
        v: [batch, timestep_v, dim_v]
        scale: 缩放因子
        attn_mask: Masking 张量 [batch, timestep_q, timestep_k]
        context: [batch, dim_v]
        """
        attention = torch.bmm(q, k.transpose(1,2))
        if scale is None:
            scale = (1 / torch.sqrt(torch.tensor(q.shape[-1], dtype=torch.float32))).item()
        attention = attention * scale
        if attn_mask:
            # 将需要 mask 的地方设为负无穷
            attention = attention.masked_fill(attn_mask, -np.inf)
        attention = torch.softmax(attention, dim=2)
        context = torch.sum(torch.bmm(attention, v), dim=1)
        return context

    def bilstm_with_mask(self, seq, seq_len, is_query=False):
        """
        desc: BiLSTM with Mask
        """
        # pack padded
        seq = pack_padded_sequence(seq, seq_len, batch_first=True, enforce_sorted=False)
        unsorted_indices = seq.unsorted_indices
        init_h = self.init_hidden()
        # bilstm
        if is_query:
            seq_out, _ = self.bilstm_query(seq, init_h)
        else:
            seq_out, _ = self.bilstm_cand(seq, init_h)
        # pad pack
        seq_out = pad_packed_sequence(seq_out, batch_first=True, padding_value=1)
        seq_out = seq_out[0][unsorted_indices]
        return seq_out

    def forward(self, query, offset, cand1, cand2, seq_len):
        # embedding
        query = self.embedding(query)
        # 拼接 mention offset：
        # 对于每个输入样本（每个query中的每个实体），将query中实体的offset位置特征编码为：长度等于句子长度，且实体部分为1，
        # 非实体部分为0的特征向量，并且拼接到每个词向量的最后一维得到 batch * seq_len * 301 维度的向量序列
        self.batch = query.shape[0]
        pos = torch.zeros([query.shape[0], query.shape[1], 1])
        for i in range(query.shape[0]):
            pos[i][offset[i]][0] = 1.0
        pos = pos.to(self.param["device"])
        query = torch.cat((query, pos), dim=2)
        # 对候选实体描述进行编码
        cand1 = self.embedding(cand1)
        cand2 = self.embedding(cand2)
        # bilstm with mask
        query_out = self.bilstm_with_mask(query, seq_len[0], is_query=True)
        cand1_out = self.bilstm_with_mask(cand1, seq_len[1])
        cand2_out = self.bilstm_with_mask(cand2, seq_len[2])
        # attention
        score_type = self.Attention(query_out, query_out, query_out)
        score_cand11 = self.Attention(query_out, cand1_out, cand1_out)
        score_cand12 = self.Attention(cand1_out, query_out, query_out)
        score_cand21 = self.Attention(query_out, cand2_out, cand2_out)
        score_cand22 = self.Attention(cand2_out, query_out, query_out)
        # 使用pairwise模型分别计算两个候选实体的得分
        score_cand1 = self.fc_rank(score_cand11 + score_cand12)
        score_cand2 = self.fc_rank(score_cand21 + score_cand22)
        # 比较两候选实体得分的
        pred_rank = torch.sigmoid(score_cand1 - score_cand2)
        # 对实体提及的类型进行预测
        pred_type = self.fc_classify(score_type)
        return pred_rank.squeeze(), pred_type

    def predict(self, query, offset, cand, seq_len):
        # embedding
        query = self.embedding(query)
        # 拼接 mention offset：方法同 self.forward
        self.batch = query.shape[0]
        pos = torch.zeros([query.shape[0], query.shape[1], 1])
        for i in range(query.shape[0]):
            pos[i][offset[i]][0] = 1.0
        pos = pos.to(self.param["device"])
        query = torch.cat((query, pos), dim=2)
        # 对候选实体描述进行编码
        cand = self.embedding(cand)
        # bilstm with mask
        query_out = self.bilstm_with_mask(query, seq_len[0], is_query=True)
        cand_out = self.bilstm_with_mask(cand, seq_len[1])
        # attention
        score_type = self.Attention(query_out, query_out, query_out)
        score_cand1 = self.Attention(query_out, cand_out, cand_out)
        score_cand2 = self.Attention(cand_out, query_out, query_out)
        # 计算该候选实体得分
        pred_rank = self.fc_rank(score_cand1 + score_cand2)
        # 预测改实体类型
        pred_type = self.fc_classify(score_type)
        return pred_rank.squeeze(), pred_type


### 3.2、模型实例化

In [ ]:
model = Model(matrix, param) # martix 是预训练的 word embedding
model.to(param["device"])

### 3.3、设置优化器、损失函数、评价指标

1. **优化器：Adadelta 优化器**  
Adadelta 是 Adagrad (Adaptive Gradient) 的改进。Adagrad是一种自适应优化方法，是自适应的为各个参数分配不同的学习率。这个学习率的变化，会受到梯度的大小和迭代次数的影响。梯度越大，学习率越小；梯度越小，学习率越大。缺点是训练后期，学习率过小，因为 Adagrad 累加之前所有的梯度平方作为分母。而 Adadelta 分母中采用距离当前时间点比较近的累计项，这可以避免在训练后期，学习率过小。

2. **损失函数：BCELoss（二分类交叉熵） + CrossEntropyLoss（多分类交叉熵）**  
MTL（muti-task Learning）中损失函数需要对每个任务的损失进行权重分配，在这个过程中，必须保证所有任务同等重要，而不能让简单任务主导整个训练过程。本案例采取简单地手动设置排序和分类任务的权重参数，此处为需要进一步改进的地方。

3. **评价指标：Accuracy**  
预测结果中预测正确的样本数与总样本数之比。准确进行实体链接的个数表示为Nr，所有实体的个数为Na：
$$
Acc = Nr/Na。  
$$
实体链接任务是预测文本中的实体的知识库id，属于预测分类的模型。一般预测分类任务的评估指标有准确率(precision)、召回率(recall)、F1值等。 

准确率定义如下：
$$
precision = \frac{|\{正确链接的待链接实体\}|}{|\{模型预测的待链接实体\}|}\\
$$

召回率定义如下：  
$$
recall = \frac{|\{正确链接的待链接实体\}|}{|\{应该被链接的待链接实体\}|}\\
$$

F1值被定义为准确率和召回率的调和评价值：
$$
F_1 = \frac{2 * precision * recall}{precision + recall}\\
$$

基于实体链接的实体消歧任务一般包括命名实体识别和实体链接两个步骤，由于命名实体识别子任务不能识别出全部的命名实体，导致召回率 recall 受到限制。因此在一般的实体消歧任务中以precision、recall、F1值为评价指标最适合。但是本案例的主要工作是把给定文本中的命名实体链接到知识库，即直接对已标注出的命名实体进行链接预测，不包含命名实体识别部分。因此上式 precision 和 recall 中的模型预测的待链接实体集合和应该被链接的集合是相同的，因此本案例中，
$$
precision = recall = Accuracy = F1
$$
综上所述，本案例以 Accuracy 为评价指标。


In [ ]:
# 优化器
optimizer = torch.optim.Adadelta(filter(lambda p: p.requires_grad, model.parameters()), lr=param["lr"])

# 联合损失函数
rank_loss_fn = nn.BCELoss()
classify_loss_fn = nn.CrossEntropyLoss()

In [15]:
# 记录预测结果
def record(result, id_list, pre_label, pre_type, label=None):
    # train
    if len(id_list[0]) == 6: 
        for i in range(len(id_list)):
            text_id, mention_id, cand1_id, cand2_id, golden_type, golden_id = id_list[i]
            if text_id not in result:
                result[text_id] = {}
            if mention_id not in result[text_id]:
                result[text_id][mention_id] ={
                    'golden_id':golden_id, 'golden_type':golden_type, 'pre_type':pre_type[i], 
                }
                if pre_label[i] - label[i] < 0.5 and pre_label[i] - label[i] > -0.5:
                    result[text_id][mention_id]['pre_id'] = golden_id
                else:
                    result[text_id][mention_id]['pre_id'] = 'NIL'
            else:
                result[text_id][mention_id]['pre_type'] += torch.clone(pre_type[i])
                if pre_label[i] - label[i] < 0.5 and pre_label[i] - label[i] > -0.5:
                    continue
                else:
                    result[text_id][mention_id]['pre_id'] = 'NIL'
    # eval or test
    elif len(id_list[0]) == 5:
        for i in range(len(id_list)):
            text_id, mention_id, cand_id, golden_type, golden_id = id_list[i]
            if text_id not in result:
                result[text_id] = {}
            if mention_id not in result[text_id]:
                result[text_id][mention_id] = {
                    'golden_id':golden_id, 'golden_type':golden_type, 'pre_id': cand_id, 
                    'pre_type':pre_type[i], 'pre_id_score': pre_label[i]
                }
            else:
                result[text_id][mention_id]['pre_type'] += torch.clone(pre_type[i])
                if pre_label[i] > result[text_id][mention_id]['pre_id_score']:
                    result[text_id][mention_id]['pre_id_score'] = pre_label[i]
                    result[text_id][mention_id]['pre_id'] = cand_id
    return result
# 计算预测结果的 Accuracy: (预测正确数 / 预测总数)
def Accuracy(result):
    right = 0
    total = 0
    for i in result.items():
        text_id = i[0]
        mentions = i[1]
        total += len(mentions)
        for j in mentions.items():
            mention_id = j[0]
            golden_id = j[1]['golden_id']
            golden_type = j[1]['golden_type']
            pre_id = j[1]['pre_id']
            pre_type = j[1]['pre_type'].argmax().item()
            if pre_id.isdigit():
                if pre_id == golden_id:
                    right += 1
            else:
                if golden_type == str(pre_type) and golden_id == 'NIL':
                    right += 1
    accuracy= right / total
    return accuracy

## 4、模型训练

（1）训练模型

（2）训练过程可视化

（3）保存模型


In [ ]:
# 生成 mention 的候选实体字典
generate_cand = GenerateCand()
if os.path.exists('./datasets/generated/cand.pkl'):
    cand_dic, ent_dic = generate_cand.load_data('kb.json')
else:
    cand_dic, ent_dic = generate_cand.interface('kb.json')

# 生成训练、验证、测试的文本数据
dataprocess = DataProcess()
if not os.path.exists('./datasets/generated/train_data.txt'):
    dataprocess.interface('train.json', cand_dic, ent_dic, is_train=True)
if not os.path.exists('./datasets/generated/dev_data.txt'):
    dataprocess.interface('dev.json', cand_dic, ent_dic, is_train=False)
if not os.path.exists('./datasets/generated/test_data.txt'):
    dataprocess.interface('test.json', cand_dic, ent_dic, is_train=False)

# matrix 向量数组；vocab 包含 vocab["w2i"]: word2idx、vocab["i2w"]：idx2word；向量维度，字词数
matrix, vocab, vec_dim, vocab_size = utils.loadWord2Vec("./datasets/pretrain_data/word2vec.iter5")

# 类型2标签字典
type2label = utils.type2label

# 数据编码
data_encoder = DataEncoder(vocab["w2i"], type2label)
if not os.path.exists('./datasets/generated/train.csv'):
    data_encoder.data_encode("./datasets/generated/train_data.txt", is_train=True)
if not os.path.exists('./datasets/generated/dev.csv'):
    data_encoder.data_encode("./datasets/generated/dev_data.txt", is_train=False)
if not os.path.exists('./datasets/generated/test.csv'):
    data_encoder.data_encode("./datasets/generated/test_data.txt", is_train=False)

# 构建数据集加载接口
train_set = DataSet("./datasets/generated/train_part.csv", is_train=True)
dev_set = DataSet("./datasets/generated/dev.csv", is_train=False)
test_set = DataSet("./datasets/generated/test.csv", is_train=False)

# dataloader
train_loader = DATA.DataLoader(train_set, batch_size=param["batch"], collate_fn=utils.collate_fn_train, drop_last=True)
dev_loader = DATA.DataLoader(dev_set, batch_size=param["batch"], collate_fn=utils.collate_fn_test, drop_last=True)
test_loader = DATA.DataLoader(test_set, batch_size=param["batch"], collate_fn=utils.collate_fn_test, drop_last=True)

### 4.1、训练模型
1. 采用小批量训练模型，每个批次输入 batch = 64 个样本，在训练集上训练 epoch = 10 轮次，每次计算模型预测输出（pre_label、pre_type）和多任务联合损失（loss），根据 loss 反向传播计算模型参数的梯度，通过 Adadelta 优化器，优化模型参数。每次在数据集上训练或评估时，通过 record 函数记录模型预测结果，通过 Accuracy 函数计算模型预测的准确率，作为模型性能的评价指标。

2. 在模型训练阶段，采用 pairwise 模型。即输入一对候选实体（cand1、cand2）比较二者与 mention 的相似度，采用 model.forward() 方法；在模型评估或测试阶段，输入 mention 和一个候选实体 cand 计算该候选实体的得分，采用 model.predict() 方法。

3. 模型在每个 epoch 设置断点，保存模型及参数。

4. 这里仅以 epoch = 1为例，硬件条件允许，可以采用多轮次训练

In [ ]:
total_step = len(train_loader)
param["epoch"] = 1
device = param["device"]
param["loss_w"] = 0.5
param["resume"] = True
train_accuracy = []
dev_accuracy = []
start_epoch = -1
# 加载断点模型
if param["resume"]:
    path_checkpoint = "./models/checkpoint/ckpt_best_0.pth"  # 断点路径
    checkpoint = torch.load(path_checkpoint)  # 加载断点
    model.load_state_dict(checkpoint['net'])  # 加载模型可学习参数
    optimizer.load_state_dict(checkpoint['optimizer'])  # 加载优化器参数
    start_epoch = checkpoint['epoch']  # 设置开始的epoch
    train_accuracy = checkpoint["train_accuracy"]
    dev_accuracy = checkpoint["test_accuracy"]
    
for epoch in range(start_epoch + 1, param["epoch"]):
    train_res = {}
    # train
    for i, data in enumerate(train_loader):
        id_list, query, offset, cand1_desc, cand2_desc, label, ent_type, seq_len = data
        # move the data to the device
        query = query.to(device)
        cand1_desc = cand1_desc.to(device)
        cand2_desc = cand2_desc.to(device)
        label = label.to(device)
        ent_type = ent_type.to(device)
        # forward
        pre_label, pre_type = model.forward(query, offset, cand1_desc, cand2_desc, seq_len)
        # loss
        rank_loss = rank_loss_fn(pre_label, label)
        type_loss = classify_loss_fn(pre_type, ent_type)
        loss = rank_loss * param["loss_w"] + type_loss * (1 - param["loss_w"])
        # optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_res = record(train_res, id_list, torch.softmax(pre_label, dim=-1), pre_type, label)
        if i % 1000 == 0:
            print('Epoch [{}/{}], Step [{}/{}]'.format(epoch, param["epoch"], i, total_step))
            print("Loss: ", loss.item(), "rank_loss: ", rank_loss.item(), "type_loss: ", type_loss.item())
    accuracy = Accuracy(train_res)
    train_accuracy.append(accuracy)
    print("train accuracy: ", accuracy)
    dev_res = {}
    # evalue
    with torch.no_grad():
        for  i, data in enumerate(dev_loader):
            id_list, query, offset, cand_desc, seq_len = data
            # move the data to the device
            query = query.to(device)
            cand_desc = cand_desc.to(device)
            # forward
            pre_label, pre_type = model.predict(query, offset, cand_desc, seq_len)
            # loss
            rank_loss = rank_loss_fn(pre_label, label)
            type_loss = classify_loss_fn(pre_type, ent_type)
            loss = rank_loss * param["loss_w"] + type_loss * (1 - param["loss_w"])
            # 记录预测结果
            dev_res = record(dev_res, id_list, torch.softmax(pre_label, dim=-1), pre_type)
        accuracy = Accuracy(dev_res)
        dev_accuracy.append(accuracy)
        print("dev accuracy: ", accuracy)
        torch.cuda.empty_cache()
    
    # 保存断点
    checkpoint = {
        "net": model.state_dict(),
        'optimizer': optimizer.state_dict(),
        "epoch": epoch,
        "train_accuracy": train_accuracy,
        "test_accuracy": dev_accuracy
    }
    if not os.path.exists("./models/checkpoint"):
        os.mkdir("./models/checkpoint")
    torch.save(checkpoint, './models/checkpoint/ckpt_best_%s.pth' % (str(epoch)))
               

In [ ]:
# del model
# import gc
# gc.collect()

## 5、测试模型 

使用测试数据中的前10条作为测试用例，载入测试用例。

（1）加载模型

（2）加载数据

（3）预测结果展示

### 5.1、加载模型

In [11]:
model = Model(matrix, param)
path_checkpoint = "./models/checkpoint/ckpt_best_0.pth"  # 断点路径
checkpoint = torch.load(path_checkpoint)  # 加载断点
model.load_state_dict(checkpoint['net'])  # 加载模型可学习参数
model.to(param["device"])

Model(
  (embedding): Embedding(636088, 300)
  (bilstm_query): LSTM(301, 150, batch_first=True, bidirectional=True)
  (bilstm_cand): LSTM(300, 150, batch_first=True, bidirectional=True)
  (fc_rank): Sequential(
    (0): Linear(in_features=300, out_features=150, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=150, out_features=1, bias=True)
    (3): Sigmoid()
  )
  (fc_classify): Sequential(
    (0): Linear(in_features=300, out_features=24, bias=True)
  )
)

### 5.2、加载数据

In [12]:
# 数据如下：
data = [
    {"text_id": "1", "text": "林平之答应岳灵珊报仇之事，从长计议，师娘想令狐冲了", "mention_data": [{"mention": "林平之", "offset": "0"}, {"mention": "岳灵珊", "offset": "5"}, {"mention": "师娘", "offset": "18"}, {"mention": "令狐冲", "offset": "21"}]},
    {"text_id": "2", "text": "思追原来是个超级妹控，不愿妹妹嫁人，然而妹妹却喜欢一博老师", "mention_data": [{"mention": "思追", "offset": "0"}, {"mention": "妹控", "offset": "8"}, {"mention": "妹妹", "offset": "13"}, {"mention": "妹妹", "offset": "20"}, {"mention": "一博", "offset": "25"}, {"mention": "老师", "offset": "27"}]},
    {"text_id": "3", "text": "铁核桃：贺韬鬼迷心窍，竟替日本特务卖命，网友：看不下去了", "mention_data": [{"mention": "铁核桃", "offset": "0"}, {"mention": "贺韬", "offset": "4"}, {"mention": "日本", "offset": "13"}, {"mention": "特务", "offset": "15"}, {"mention": "网友", "offset": "20"}]},
    {"text_id": "4", "text": "经典动漫：我想笑容一定很适合你", "mention_data": [{"mention": "动漫", "offset": "2"}, {"mention": "笑容", "offset": "7"}]},
    {"text_id": "5", "text": "外来媳妇本地郎：阿宗给八哥滴血认亲，为了不被恨竟扮成女人！", "mention_data": [{"mention": "外来媳妇本地郎", "offset": "0"}, {"mention": "阿宗", "offset": "8"}, {"mention": "八哥", "offset": "11"}, {"mention": "滴血认亲", "offset": "13"}, {"mention": "女人", "offset": "26"}]},
    {"text_id": "6", "text": "儿子祝融被杀害，西天王大发雷霆，立即下令捉拿天庭三公主", "mention_data": [{"mention": "儿子", "offset": "0"}, {"mention": "祝融", "offset": "2"}, {"mention": "西天王", "offset": "8"}, {"mention": "天庭三公主", "offset": "22"}]},
    {"text_id": "7", "text": "巨神战击队：仇恨怪先发动攻击，一打二你也很嚣张！", "mention_data": [{"mention": "巨神战击队", "offset": "0"}, {"mention": "仇恨怪", "offset": "6"}, {"mention": "嚣张", "offset": "21"}]},
    {"text_id": "8", "text": "俩孩子青梅竹马两小无猜，父亲为拆散两人，竟狠下心来这样做", "mention_data": [{"mention": "孩子", "offset": "1"}, {"mention": "父亲", "offset": "12"}, {"mention": "人", "offset": "18"}]},
    {"text_id": "9", "text": "海胆肆无忌惮，吓到了蟹老板，章鱼哥和海绵宝宝！", "mention_data": [{"mention": "海胆", "offset": "0"}, {"mention": "蟹老板", "offset": "10"}, {"mention": "章鱼哥", "offset": "14"}, {"mention": "海绵宝宝", "offset": "18"}]},
    {"text_id": "10", "text": "太贱了，宝哥pk李永利用语音说话分散注意，永哥：你不要干扰我", "mention_data": [{"mention": "宝哥", "offset": "4"}, {"mention": "李永", "offset": "8"}, {"mention": "语音", "offset": "12"}, {"mention": "永哥", "offset": "21"}]},
]

In [17]:
# 生成 mention 的候选实体字典
generate_cand = GenerateCand()
if os.path.exists('./datasets/generated/cand.pkl'):
    cand_dic, ent_dic = generate_cand.load_data('kb.json')
else:
    cand_dic, ent_dic = generate_cand.interface('kb.json')

# 生成训练、验证、测试的文本数据
dataprocess = DataProcess()
dataprocess.interface('test_example.json', cand_dic, ent_dic, is_train=False)

# 数据编码
data_encoder = DataEncoder(vocab["w2i"], type2label)
data_encoder.data_encode("./datasets/generated/test_example_data.txt", is_train=False)

# 构建数据集加载接口
test_set = DataSet("./datasets/generated/test_example.csv", is_train=False)

# dataloader
test_loader = DATA.DataLoader(test_set, batch_size=2, collate_fn=collate_fn_test)

Encode test_example_data: 327it [00:00, 1022.56it/s]


### 5.3、预测结果展示

In [19]:
result = {}
for  i, test_data in enumerate(test_loader):
    id_list, query, offset, cand_desc, seq_len = test_data
    # move the data to the device
    # query = query.to(param['device'])
    # cand_desc = cand_desc.to(param['device'])
    # forward
    pre_label, pre_type = model.predict(query, offset, cand_desc, seq_len)
    # 记录预测结果
    result = record(result, id_list, torch.softmax(pre_label, dim=-1), pre_type)

# 展示预测结果
for i in result.items():
    try: 
        text_id = i[0]
        mentions = i[1]
        mention_data = data[int(text_id)]["mention_data"]
        # print(mention_data)
        for j in mentions.items():
            try: 
                mention_id = int(j[0])
                mention_data[mention_id]["pre_id"] = j[1]['pre_id']
                mention_data[mention_id]["pre_type"] = j[1]['pre_type'].argmax().item()
            except:
                continue
        data[int(text_id)]["mention_data"] = mention_data
    except:
        continue
for item in data:
    print(item)

# 保存预测结果
with open("./results/result_test_example.json", 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False) 

{'text_id': '1', 'text': '林平之答应岳灵珊报仇之事，从长计议，师娘想令狐冲了', 'mention_data': [{'mention': '林平之', 'offset': '0'}, {'mention': '岳灵珊', 'offset': '5'}, {'mention': '师娘', 'offset': '18'}, {'mention': '令狐冲', 'offset': '21'}]}
{'text_id': '2', 'text': '思追原来是个超级妹控，不愿妹妹嫁人，然而妹妹却喜欢一博老师', 'mention_data': [{'mention': '思追', 'offset': '0', 'pre_id': '146294', 'pre_type': 0}, {'mention': '妹控', 'offset': '8', 'pre_id': 'NIL', 'pre_type': 0}, {'mention': '妹妹', 'offset': '13', 'pre_id': '269962', 'pre_type': 0}, {'mention': '妹妹', 'offset': '20', 'pre_id': 'NIL', 'pre_type': 0}, {'mention': '一博', 'offset': '25'}, {'mention': '老师', 'offset': '27'}]}
{'text_id': '3', 'text': '铁核桃：贺韬鬼迷心窍，竟替日本特务卖命，网友：看不下去了', 'mention_data': [{'mention': '铁核桃', 'offset': '0', 'pre_id': '46770', 'pre_type': 0}, {'mention': '贺韬', 'offset': '4', 'pre_id': 'NIL', 'pre_type': 0}, {'mention': '日本', 'offset': '13', 'pre_id': 'NIL', 'pre_type': 0}, {'mention': '特务', 'offset': '15', 'pre_id': 'NIL', 'pre_type': 0}, {'mention': '网友', 'offset'